# Searching the EOPF Sentinel Zarr Samples Service STAC API

## Introduction

In this comprehensive tutorial, we will explore how to **programmatically** access Sentinel-1, Sentinel-2, and Sentinel-3 `.zarr` collections available through the [EOPF Sentinel Zarr Sample Service STAC](https://stac.browser.user.eopf.eodc.eu/?.language=en).

This powerful API provides a structured way to search and access EOPF data using Python, offering the same functionality you see in the web interface but with programmatic control.

### What You'll Learn
- 🔍 How to **programmatically browse** available collections
- 📊 Understanding **collection metadata** in user-friendly terms
- 🎯 **Searching for specific data** with practical examples
- 🔗 How **API requests relate** to the web interface
- 💪 **Hands-on exercises** to practice your skills

### Understanding STAC
**STAC (SpatioTemporal Asset Catalog)** is a specification that provides a common language to describe geospatial information. Think of it as a standardized way to organize and discover Earth observation data - like a library catalog for satellite imagery!

#### Prerequisites

The `pystac` and `pystac_client` libraries are essential for requesting and conducting deep searches within the STAC environment, enabling efficient data processing. We encourage you to consult the [pystac documentation](https://pystac.readthedocs.io/en/stable/) and [pystac_client documentation](https://pystac-client.readthedocs.io/en/latest/api.html) for additional resources and in-depth information.

> **Note:** <br>
> We recommend creating a virtual environment as it helps manage library versions and prevents conflicts with other Python projects on your system. <br>
> Follow [this tutorial](), to create a virtual environment that will allow us to run all the available tutorials in [EOPF-101](https://github.com/eopf-toolkit/eopf-101). 


<hr>


##### Import libraries
To ensure a stable and reproducible environment for our project, we begin by setting up our dependencies.

In [30]:
import requests
from typing import List, Optional, cast
from pystac import Collection, MediaType
from pystac_client import Client, CollectionClient
from datetime import datetime
import folium
from folium import plugins
import json
from shapely.geometry import shape, box
import random

##### Helper functions

##### `list_found_elements`


As we anticipate visualising several elements stored in lists, we will define a helper function to streamline id's retrieval inside our workflow.

In [31]:
def list_found_elements(search_result):
    """Extract item IDs and collection IDs from search results."""
    id = []
    coll = []
    for item in search_result.items(): #retrieves the result inside the catalogue.
        id.append(item.id)    # stores item (image) id
        coll.append(item.collection_id)  #stores the collections id
    return id , coll

def display_search_results_summary(search_result, search_description=""):
    """Display a comprehensive summary of search results."""
    ids, collections = list_found_elements(search_result)
    unique_collections = set(collections)
    
    print(f"🔍 Search Results {search_description}:")
    print(f"   📊 Total Items Found: {len(ids)}")
    print(f"   🛰️  Collections Represented: {len(unique_collections)}")
    print(f"   📋 Collection Names: {', '.join(sorted(unique_collections))}")
    
    # Show collection distribution if multiple collections
    if len(unique_collections) > 1:
        collection_counts = {}
        for coll in collections:
            collection_counts[coll] = collection_counts.get(coll, 0) + 1
        
        print("\n   📈 Items per Collection:")
        for coll, count in sorted(collection_counts.items()):
            print(f"      • {coll}: {count} items")
    print()

def explain_search_parameters():
    """Explain the different search parameters available."""
    print("🔧 Available Search Parameters:\n")
    
    explanations = {
        "bbox": "📍 **Bounding Box** - Define a rectangular area using [min_lon, min_lat, max_lon, max_lat]",
        "datetime": "📅 **Date/Time** - Filter by acquisition time using ISO 8601 format or date ranges",
        "collections": "🛰️  **Collections** - Specify which satellite missions/products to search",
        "limit": "📊 **Limit** - Maximum number of results to return (default: 10)",
        "query": "🔍 **Query** - Advanced filtering using collection-specific properties"
    }
    
    for param, explanation in explanations.items():
        print(f"{explanation}")
        print()

def display_collection_metadata_guide():
    """Explain what collection metadata means in user-friendly terms."""
    print("📚 Understanding Collection Metadata:\n")
    
    metadata_guide = {
        "Collection ID": "🆔 Unique identifier for the dataset (e.g., 'sentinel-2-l2a')",
        "Description": "📝 What type of data this collection contains",
        "Temporal Extent": "📅 Time period covered by the data (from first to last acquisition)",
        "Spatial Extent": "🌍 Geographic area covered by the collection",
        "Keywords": "🏷️  Tags that help categorize the data type",
        "License": "📄 Usage rights and restrictions for the data"
    }
    
    for term, explanation in metadata_guide.items():
        print(f"{explanation}")
        print()
    
    print("💡 **Why This Matters**: Understanding metadata helps you quickly identify if a collection contains the type of data you need for your research or project!")

def create_search_results_map(search_result, search_description="", center_lat=None, center_lon=None, zoom_start=6):
    """Create an interactive map showing the footprints of search results."""
    
    # Collection color mapping for different satellite missions
    collection_colors = {
        'sentinel-1-grd': '#FF6B6B',        # Red for SAR
        'sentinel-2-l2a': '#4ECDC4',        # Teal for optical
        'sentinel-2-l1c': '#45B7D1',        # Blue for optical L1C
        'sentinel-3-olci-l2-lfr': '#96CEB4', # Green for ocean color
        'sentinel-3-slstr-l2-lst': '#FFEAA7', # Yellow for land surface temp
        'sentinel-3-synergy': '#DDA0DD',     # Purple for synergy
        'default': '#95A5A6'                # Gray for others
    }
    
    # Get items from search results
    items = list(search_result.items())
    
    if not items:
        print("⚠️  No items found to display on map.")
        return None
    
    # Calculate center if not provided
    if center_lat is None or center_lon is None:
        # Use the first item's centroid as center
        first_item = items[0]
        if hasattr(first_item, 'geometry') and first_item.geometry:
            geom = shape(first_item.geometry)
            center_lon, center_lat = geom.centroid.x, geom.centroid.y
        else:
            center_lat, center_lon = 50.0, 10.0  # Default to Europe
    
    # Create the map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=zoom_start,
        tiles='OpenStreetMap'
    )
    
    # Add alternative tile layers
    folium.TileLayer('Stamen Terrain', name='Terrain', attr='Map tiles by Stamen Design, under CC BY 3.0. Data by OpenStreetMap, under ODbL.').add_to(m)
    folium.TileLayer('CartoDB positron', name='Light', attr='Map tiles by CartoDB, under CC BY 3.0. Data by OpenStreetMap, under ODbL.').add_to(m)
    
    # Group items by collection for legend
    collection_counts = {}
    
    # Add each item's footprint to the map
    for i, item in enumerate(items):
        if not hasattr(item, 'geometry') or not item.geometry:
            continue
            
        collection_id = item.collection_id
        collection_counts[collection_id] = collection_counts.get(collection_id, 0) + 1
        
        # Get color for this collection
        color = collection_colors.get(collection_id, collection_colors['default'])
        
        # Create popup content with item metadata
        popup_content = f"""
        <div style="width: 300px;">
            <h4>🛰️ {item.id}</h4>
            <p><strong>Collection:</strong> {collection_id}</p>
            <p><strong>Date:</strong> {item.datetime.strftime('%Y-%m-%d %H:%M:%S') if item.datetime else 'N/A'}</p>
        """
        
        # Add cloud cover if available
        if hasattr(item, 'properties') and 'eo:cloud_cover' in item.properties:
            cloud_cover = item.properties['eo:cloud_cover']
            popup_content += f"<p><strong>☁️ Cloud Cover:</strong> {cloud_cover:.1f}%</p>"
        
        popup_content += "</div>"
        
        # Add the footprint to the map
        folium.GeoJson(
            item.geometry,
            style_function=lambda feature, color=color: {
                'fillColor': color,
                'color': color,
                'weight': 2,
                'fillOpacity': 0.3,
                'opacity': 0.8
            },
            popup=folium.Popup(popup_content, max_width=300),
            tooltip=f"{item.id} ({collection_id})"
        ).add_to(m)
    
    # Add a custom legend
    legend_html = f'''
    <div style="position: fixed; 
                top: 10px; right: 10px; width: 200px; height: auto; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:14px; padding: 10px">
    <h4>🔍 Search Results {search_description}</h4>
    <p><strong>Total Items:</strong> {len(items)}</p>
    <p><strong>Collections:</strong></p>
    '''
    
    for collection_id, count in sorted(collection_counts.items()):
        color = collection_colors.get(collection_id, collection_colors['default'])
        legend_html += f'<p><span style="color: {color};">●</span> {collection_id}: {count}</p>'
    
    legend_html += '</div>'
    
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    print(f"🗺️ Created interactive map with {len(items)} item footprints")
    print(f"📊 Collections represented: {', '.join(collection_counts.keys())}")
    
    return m

def visualize_search_area(bbox, area_name="Search Area"):
    """Create a map showing the search bounding box."""
    min_lon, min_lat, max_lon, max_lat = bbox
    
    # Calculate center
    center_lat = (min_lat + max_lat) / 2
    center_lon = (min_lon + max_lon) / 2
    
    # Create map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=10,
        tiles='OpenStreetMap'
    )
    
    # Add bounding box rectangle
    folium.Rectangle(
        bounds=[[min_lat, min_lon], [max_lat, max_lon]],
        color='red',
        fill=True,
        fillColor='red',
        fillOpacity=0.2,
        popup=f"Search Area: {area_name}",
        tooltip=f"Bounding Box: {bbox}"
    ).add_to(m)
    
    # Add corner markers
    folium.Marker(
        [min_lat, min_lon],
        popup=f"SW Corner: {min_lat:.4f}, {min_lon:.4f}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)
    
    folium.Marker(
        [max_lat, max_lon],
        popup=f"NE Corner: {max_lat:.4f}, {max_lon:.4f}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)
    
    return m

<hr>

## API connection

Our first step is to construct our request to interact with the EOPF STAC API. This involves defining the parameters for the data we wish to retrieve.<br>
The API's base URL is conveniently available through the [OpenAPI service description](https://stac.core.eopf.eodc.eu), which can be found in the **Additional Resources** tab of the [EOPF Sentinel Zarr Sample Service STAC API](https://stac.browser.user.eopf.eodc.eu/?.language=en).

![eopf_stac_api_url.png](./img/eopf_stac_api_url.png)

This entry, provides the starting point of the Catalogue.

In [32]:
max_description_length = 100

eopf_stac_api_root_endpoint = "https://stac.core.eopf.eodc.eu/" #root starting point
client = Client.open(url=eopf_stac_api_root_endpoint)

Rectifying the catalog we have just accessed:

In [33]:
print(
    "Connected to Catalog {id}: {description}".format(
        id=client.id,
        description=client.description
        if len(client.description) <= max_description_length
        else client.description[: max_description_length - 3] + "...",
    )
)

Connected to Catalog eopf-sample-service-stac-api: STAC catalog of the EOPF Sentinel Zarr Samples Service


It is important to remember that the Sentinel Zarr Sample Service STAC **is actively under development** and receives continuous updates and additions to its collections. To ensure we access only currently available resources, we include a verification step to confirm data availability within the catalogue.<br> This proactive approach helps us understand what data is presently accessible.

> **Note:** <br>
> To explore further issues or more considerations check un the [EOPF Sentinel Zarr Samples Service](https://zarr.eopf.copernicus.eu/) updates and their [Github Issues](https://github.com/EOPF-Sample-Service/eopf-stac/issues)

In [34]:
all_collections: Optional[List[Collection]] = None
# The simplest approach to retrieve all collections may fail due to #18 on Github.

try:
    all_collections = [_ for _ in client.get_all_collections()]
    print(
        "* [https://github.com/EOPF-Sample-Service/eopf-stac/issues/18 appears to be resolved]"
    )
except Exception:
    print(
        "* [https://github.com/EOPF-Sample-Service/eopf-stac/issues/18 appears to not be resolved]"
    )


* [https://github.com/EOPF-Sample-Service/eopf-stac/issues/18 appears to not be resolved]


We observe that one collection is currently undergoing updates, and a refinement of our search to work exclusively with available resources is needed.

## Available Collections

We can filter the available Collections to have an overview of the instances we can retrieve:

In [35]:
if all_collections is None:
    # If collection retrieval fails due to #18.
    valid_collections: List[Collection] = []
    for collection_href in [link.absolute_href for link in client.get_child_links()]:
        collection_dict = requests.get(url=collection_href).json()
        try:
            # Attempt to retrieve collections individually.
            valid_collections.append(Collection.from_dict(collection_dict))
        except Exception as e:
            if isinstance(e, TypeError) and "not subscriptable" in str(e).lower():
                # This exception is expected for some collections due to #18.
                continue
            else:
                raise e
            
    all_collections = valid_collections


And the available collections that can be explored are:

In [36]:
print(all_collections)

[<Collection id=sentinel-2-l2a>, <Collection id=sentinel-3-slstr-l1-rbt>, <Collection id=sentinel-3-olci-l2-lfr>, <Collection id=sentinel-2-l1c>, <Collection id=sentinel-3-slstr-l2-lst>, <Collection id=sentinel-1-l1-slc>, <Collection id=sentinel-3-olci-l1-efr>, <Collection id=sentinel-3-olci-l1-err>, <Collection id=sentinel-1-l2-ocn>, <Collection id=sentinel-1-l1-grd>]


### 📚 Understanding Collection Metadata

Before we dive into the collection details, let's understand what the metadata means:

In [37]:
# Display the metadata guide to help users understand what they're seeing
display_collection_metadata_guide()

📚 Understanding Collection Metadata:

🆔 Unique identifier for the dataset (e.g., 'sentinel-2-l2a')

📝 What type of data this collection contains

📅 Time period covered by the data (from first to last acquisition)

🌍 Geographic area covered by the collection

🏷️  Tags that help categorize the data type

📄 Usage rights and restrictions for the data

💡 **Why This Matters**: Understanding metadata helps you quickly identify if a collection contains the type of data you need for your research or project!


Now let's examine the actual collections available in the EOPF STAC catalog:

In [38]:
for collection in all_collections:
    collection_parent = collection.get_parent()
    start_date = collection.extent.temporal.intervals[0][0] # Get the first available date of the items
    end_date = collection.extent.temporal.intervals[0][1]   # Get the last available date of the items
    print("Collection {id}".format(id=collection.id))  # Collection id
    print(
        " - Description: {description}".format(        # Summary of the contained information
            description=collection.description
            if len(collection.description) <= max_description_length
            else collection.description[: max_description_length - 3] + "..."
        )
    )
    print(
        " - Temporal Extent: {start_date} to {end_date}".format(
        start_date = start_date.strftime("%Y-%m-%d"),
        end_date = end_date.strftime("%Y-%m-%d")
        )
    )
    

Collection sentinel-2-l2a
 - Description: The Sentinel-2 Level-2A Collection 1 product provides orthorectified Surface Reflectance (Bottom-...
 - Temporal Extent: 2018-06-01 to 2025-06-11
Collection sentinel-3-slstr-l1-rbt
 - Description: The Sentinel-3 SLSTR Level-1B RBT product provides radiances and brightness temperatures for each...
 - Temporal Extent: 2025-04-28 to 2025-06-11
Collection sentinel-3-olci-l2-lfr
 - Description: The Sentinel-3 OLCI L2 LFR product provides land and atmospheric geophysical parameters computed ...
 - Temporal Extent: 2025-04-28 to 2025-06-11
Collection sentinel-2-l1c
 - Description: The Sentinel-2 Level-1C product is composed of 110x110 km2 tiles (ortho-images in UTM/WGS84 proje...
 - Temporal Extent: 2025-01-13 to 2025-06-11
Collection sentinel-3-slstr-l2-lst
 - Description: The Sentinel-3 SLSTR Level-2 LST product provides land surface temperature.
 - Temporal Extent: 2025-04-28 to 2025-06-11
Collection sentinel-1-l1-slc
 - Description: The Sentinel-1

## 🔍 Understanding Search Parameters

Before we start searching, let's understand the different ways we can filter and find data:

In [39]:
# Display explanation of search parameters
explain_search_parameters()

🔧 Available Search Parameters:

📍 **Bounding Box** - Define a rectangular area using [min_lon, min_lat, max_lon, max_lat]

📅 **Date/Time** - Filter by acquisition time using ISO 8601 format or date ranges

🛰️  **Collections** - Specify which satellite missions/products to search

📊 **Limit** - Maximum number of results to return (default: 10)

🔍 **Query** - Advanced filtering using collection-specific properties



## 🗺️ Practical Search Examples

Now let's put these search parameters into practice with real examples. Each example demonstrates how the API calls relate to what you would do in the web interface.

**💡 Web Interface Connection**: When you use the STAC browser in your web browser, it's making the same API calls we're about to make programmatically!

### Example 1: Spatial Search with Bounding Box

Let's search for data over a specific area. We'll use Innsbruck, Austria as our example.

**💡 Web Interface Equivalent**: This is the same as drawing a rectangle on the map in the STAC browser!

**Understanding Bounding Box**: A bounding box defines a rectangular area using coordinates: `[min_longitude, min_latitude, max_longitude, max_latitude]`

In [40]:
# Define our area of interest: Innsbruck, Austria
innsbruck_bbox = (11.124756, 47.311058, 11.459839, 47.463624)  # [min_lon, min_lat, max_lon, max_lat]

print("🗺️  Searching for data over Innsbruck, Austria...")
print(f"📍 Bounding Box: {innsbruck_bbox}")
print(f"   • Southwest Corner: {innsbruck_bbox[0]:.4f}°, {innsbruck_bbox[1]:.4f}°")
print(f"   • Northeast Corner: {innsbruck_bbox[2]:.4f}°, {innsbruck_bbox[3]:.4f}°")

# Perform the search
bbox_search = client.search(bbox=innsbruck_bbox, collections=['sentinel-1-l1-slc', 'sentinel-1-l1-grd', 'sentinel-2-l2a'], limit=10)

🗺️  Searching for data over Innsbruck, Austria...
📍 Bounding Box: (11.124756, 47.311058, 11.459839, 47.463624)
   • Southwest Corner: 11.1248°, 47.3111°
   • Northeast Corner: 11.4598°, 47.4636°


In [41]:
# Display results using our enhanced helper function
display_search_results_summary(bbox_search, "for Innsbruck area")

🔍 Search Results for Innsbruck area:
   📊 Total Items Found: 64
   🛰️  Collections Represented: 3
   📋 Collection Names: sentinel-1-l1-grd, sentinel-1-l1-slc, sentinel-2-l2a

   📈 Items per Collection:
      • sentinel-1-l1-grd: 29 items
      • sentinel-1-l1-slc: 1 items
      • sentinel-2-l2a: 34 items



**🔍 Analysis**: The search results show us which satellite collections have data covering our area of interest. This gives us a clear picture of the data density and variety available for our Area of Interest (AOI).

### 🗺️ Visualizing Search Results on a Map

Now let's create an interactive map to visualize the footprints of our search results. This helps us understand the spatial distribution of available data.

In [42]:
# First, let's visualize our search area
print("📍 Visualizing the search area:")
search_area_map = visualize_search_area(innsbruck_bbox, "Innsbruck, Austria")
search_area_map

📍 Visualizing the search area:


In [43]:
# Now let's create a map showing the footprints of all found items
print("🛰️ Creating map with satellite data footprints:")
results_map = create_search_results_map(
    bbox_search, 
    "for Innsbruck area",
    center_lat=47.3, 
    center_lon=11.4, 
    zoom_start=8
)

results_map

🛰️ Creating map with satellite data footprints:
🗺️ Created interactive map with 64 item footprints
📊 Collections represented: sentinel-1-l1-grd, sentinel-2-l2a, sentinel-1-l1-slc


**🎨 Understanding the Map**:
- **Different colors** represent different satellite collections (Sentinel-1, Sentinel-2, etc.)
- **Polygons** show the actual footprints of satellite acquisitions
- **Click on any footprint** to see detailed metadata (date, cloud cover, etc.)
- **Layer control** (top right) lets you switch between different map styles
- **Legend** (top right) shows the count of items per collection

This visualization helps you understand:
- **Spatial coverage**: How well your area is covered by different satellites
- **Data density**: Areas with more or fewer observations
- **Mission overlap**: Where different Sentinel missions provide complementary data

### Time frame

Filtering data by a specific time interval is also incredibly useful. The `datetime` parameter allows us to focus on imagery captured within a particular period.<br>
Let's define an interval that spans, for example, between May 1, 2020, and May 31, 2023.

In [44]:
time_frame = client.search(
    datetime="2020-05-01T00:00:00Z/2023-05-31T23:59:59.999999Z")


Then, the available collections and number of assets contained are:

In [45]:
time_items=list_found_elements(time_frame) #we apply our constructed function
print('Available Collections: ',set(time_items[1]))
print('Retrieved Items: ',len(time_items[0]))

Available Collections:  {'sentinel-2-l2a', 'sentinel-1-l1-grd'}
Retrieved Items:  270


### Combined Search

Now, we can explore how to refine our search even further by combining multiple criteria. <br>
This capability is incredibly powerful for pinpointing precisely the data we need. For instance, we can search for items within a specific time frame and from a particular collection simultaneously.<br>
We will focus our attention on the Sentinel-2 L2A Collection.

We define the `collections` argument by the collection `id` argument inside the Catalogue:

In [46]:
sentinel2 = client.search(
    collections= ['sentinel-2-l2a'], # the collection we are interesed in
    datetime="2020-05-01T00:00:00Z/2023-05-31T23:59:59.999999Z" # the time frame of interest
)


We can then observe that the entire Sentinel-2 L2A collection, up to our selected timeframe, comprises:

In [47]:
multiple_items=list_found_elements(sentinel2) #we apply our constructed function
print('Retrieved Items between 01-May-2020 and 31-May-2023: ',len(multiple_items[0]))

Retrieved Items between 01-May-2020 and 31-May-2023:  196


A common workflow in Earth Observation (EO) analysis involves retrieving datasets within a defined AOI and a specific time frame.<br>
`pystac` allows us to combine these three arguments seamlessly during our search.

In [48]:
innsbruck_s2 = client.search(
    bbox=(11.124756, 47.311058, # AOI extent
          11.459839,47.463624),
    collections= ['sentinel-2-l2a'], # interest Collection
    datetime='2020-05-01T00:00:00Z/2025-05-31T23:59:59.999999Z' # interest period
)

So, for Innsbruck outskirts area, we have:

In [49]:
new_ins=list_found_elements(innsbruck_s2)
print('Retrieved Sentinel 2 L2A Items between 01-May-2020 and 31-May-2025 close to Innsbruck, Austria: ',len(new_ins[0]))

Retrieved Sentinel 2 L2A Items between 01-May-2020 and 31-May-2025 close to Innsbruck, Austria:  27


And we can do the same with another location. <br>

We set a new AOI as the Sea Area of Rostock, Germany with the `bbox`:

In [50]:
rostock_s2 = client.search(
    bbox=(11.766357,53.994566, # AOI extent
          12.332153,54.265086),
    collections= ['sentinel-2-l2a'], # interest Collection
    datetime='2020-05-01T00:00:00Z/2025-05-31T23:59:59.999999Z' # interest period
)

new_ins=list_found_elements(rostock_s2)
print('Retrieved Sentinel 2 L2A Items between 01-May-2020 and 31-May-2025 close to Rostock, Germany: ',len(new_ins[0]))

Retrieved Sentinel 2 L2A Items between 01-May-2020 and 31-May-2025 close to Rostock, Germany:  124


### Location in Catalogue

Thus far, we have conducted a search within the STAC catalogue and browsed the general metadata of the collections. To access the actual `zarr` items themselves, we need to locate the object that allows us to explore their cloud storage location.<br>
Such an element can be found within the `.self_href` attribute when retrieving the selected items (searched through `client.search()`). <br>

Defining our search again in Innsbruck:

In [51]:
new_ins=list_found_elements(innsbruck_s2)
print('Retrieved Sentinel 2 L2A Items between 01-May-2020 and 31-May-2025 close to Innsbruck, Austria: ',len(new_ins[0]))

Retrieved Sentinel 2 L2A Items between 01-May-2020 and 31-May-2025 close to Innsbruck, Austria:  27


In [52]:
c_sentinel2 = client.get_collection('sentinel-2-l2a')
c_sentinel2_urls=[]
for x in range(len(new_ins[0])):
    c_sentinel2_urls.append(c_sentinel2.get_item(new_ins[0][x]).self_href) # call out, the defined search at Innsbruck, Austria.

We are able to retrieve the location of the metadata `.json` extension, which contains the individual parameters for each asset of the item. <br>
This information will allow us to obtain the items' metadata, such as the `hrefs` (the cloud storage path) for accessing each of the assets that comprise the `.zarr` item or group of items of our interest.


In [53]:
c_sentinel2_urls[:4]

['https://stac.core.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2B_MSIL2A_20250530T101559_N0511_R065_T32TPT_20250530T130924',
 'https://stac.core.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2A_MSIL2A_20250527T102041_N0511_R065_T32TPT_20250527T165916',
 'https://stac.core.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2B_MSIL2A_20250527T100559_N0511_R022_T32TPT_20250527T155229',
 'https://stac.core.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2C_MSIL2A_20250525T101621_N0511_R065_T32TPT_20250525T153015']

## 💪 Hands-on Exercises

Now it's time to practice what you've learned! These exercises will help you master the STAC API and understand how to find the data you need.

### 🎯 Exercise 1: Explore Your Own Area of Interest

**Your Task**: 
1. Go to [http://bboxfinder.com/](http://bboxfinder.com/) and select an area of interest (your hometown, a research site, etc.)
2. Copy the bounding box coordinates
3. Use the code template below to search for data over your area

**Code Template**:

In [54]:
# Exercise 1: Your area of interest
# Replace these coordinates with your bounding box from bboxfinder.com
my_bbox = (0.0, 0.0, 1.0, 1.0)  # [min_lon, min_lat, max_lon, max_lat]
my_location_name = "My Location"  # Give your location a name

print(f"🗺️  Searching for data over {my_location_name}...")
print(f"📍 Bounding Box: {my_bbox}")

# Perform the search
my_search = client.search(bbox=my_bbox)

# Display results
display_search_results_summary(my_search, f"for {my_location_name}")

🗺️  Searching for data over My Location...
📍 Bounding Box: (0.0, 0.0, 1.0, 1.0)
🔍 Search Results for My Location:
   📊 Total Items Found: 63
   🛰️  Collections Represented: 7
   📋 Collection Names: sentinel-1-l1-grd, sentinel-1-l2-ocn, sentinel-3-olci-l1-efr, sentinel-3-olci-l1-err, sentinel-3-olci-l2-lfr, sentinel-3-slstr-l1-rbt, sentinel-3-slstr-l2-lst

   📈 Items per Collection:
      • sentinel-1-l1-grd: 2 items
      • sentinel-1-l2-ocn: 2 items
      • sentinel-3-olci-l1-efr: 2 items
      • sentinel-3-olci-l1-err: 48 items
      • sentinel-3-olci-l2-lfr: 3 items
      • sentinel-3-slstr-l1-rbt: 3 items
      • sentinel-3-slstr-l2-lst: 3 items



### 🎯 Exercise 2: Temporal Analysis by Year

**Your Task**: Compare data availability across different years (2022, 2023, 2024)

**Code Template**:

In [56]:
# Exercise 2: Temporal analysis
years_to_analyze = [2022, 2023]

print("📅 Analyzing data availability by year:\n")

for year in years_to_analyze:
    # Define the time range for the year
    start_date = f"{year}-01-01T00:00:00Z"
    end_date = f"{year}-12-31T23:59:59Z"
    datetime_range = f"{start_date}/{end_date}"
    
    # Search for data in that year
    year_search = client.search(datetime=datetime_range)
    
    # Display results
    print(f"📊 Year {year}:")
    display_search_results_summary(year_search, f"for {year}")
    print("-" * 50)

📅 Analyzing data availability by year:

📊 Year 2022:
🔍 Search Results for 2022:
   📊 Total Items Found: 147
   🛰️  Collections Represented: 2
   📋 Collection Names: sentinel-1-l1-grd, sentinel-2-l2a

   📈 Items per Collection:
      • sentinel-1-l1-grd: 37 items
      • sentinel-2-l2a: 110 items

--------------------------------------------------
📊 Year 2023:
🔍 Search Results for 2023:
   📊 Total Items Found: 75
   🛰️  Collections Represented: 3
   📋 Collection Names: sentinel-1-l1-grd, sentinel-1-l1-slc, sentinel-2-l2a

   📈 Items per Collection:
      • sentinel-1-l1-grd: 37 items
      • sentinel-1-l1-slc: 2 items
      • sentinel-2-l2a: 36 items

--------------------------------------------------


### 🎯 Exercise 3: Mission-Specific Analysis

**Your Task**: Choose a specific Sentinel mission and analyze its data availability

**Available Collections** (choose one):
- `sentinel-1-grd`
- `sentinel-2-l2a`
- `sentinel-3-olci-l2-lfr`
- `sentinel-3-slstr-l2-lst`

**Code Template**:

In [59]:
# Exercise 3: Mission-specific analysis
# Choose your mission (uncomment one line below)
# chosen_mission = 'sentinel-1-grd'        # SAR data
# chosen_mission = 'sentinel-2-l2a'        # Optical data
# chosen_mission = 'sentinel-3-olci-l2-lfr'  # Ocean color
chosen_mission = 'sentinel-2-l2a'  # Default choice

print(f"🛰️  Analyzing {chosen_mission} mission data...\n")

# Search for all data from this mission
mission_search = client.search(
    collections=[chosen_mission],
    datetime="2020-01-01T00:00:00Z/2023-12-31T23:59:59Z"
)

display_search_results_summary(mission_search, f"for {chosen_mission}")

# Bonus: Combine with your area of interest
print("🎯 Bonus: Combining mission with your area of interest...")
combined_search = client.search(
    bbox=my_bbox,  # Using the bbox from Exercise 1
    collections=[chosen_mission],
    datetime="2020-01-01T00:00:00Z/2023-12-31T23:59:59Z"
)

display_search_results_summary(combined_search, f"for {chosen_mission} over {my_location_name}")

🛰️  Analyzing sentinel-2-l2a mission data...

🔍 Search Results for sentinel-2-l2a:
   📊 Total Items Found: 232
   🛰️  Collections Represented: 1
   📋 Collection Names: sentinel-2-l2a

🎯 Bonus: Combining mission with your area of interest...
🔍 Search Results for sentinel-2-l2a over My Location:
   📊 Total Items Found: 0
   🛰️  Collections Represented: 0
   📋 Collection Names: 



### 🤔 Reflection Questions

After completing the exercises, consider these questions:

1. **Data Availability**: Which year had the most data available? Why might this be?

2. **Geographic Patterns**: How does data availability vary between different locations? What factors might influence this?

3. **Mission Differences**: How do the different Sentinel missions compare in terms of data volume and coverage?

4. **API vs Web Interface**: How does using the programmatic API compare to browsing the web interface? What are the advantages of each approach?

**💡 Pro Tip**: Try combining different search parameters to create more specific queries for your research needs!

<hr>

## Conclusion

This tutorial has provided a clear and practical introduction to exploring the [EOPF Sentinel Zarr Sample Service STAC API](https://stac.browser.user.eopf.eodc.eu/?.language=en).<br>
We were able to explore how to connect to the EOPF available API, navigate its structure, and filter data by spatial and temporal criteria though Python. <br>

By leveraging the `pystac` and `pystac_client` libraries, we have the tools to efficiently search for and access the vast amounts of Earth Observation data available through this powerful catalog. <br>
This understanding forms a solid foundation for further analysis and application of Sentinel data in your projects.

#### What's next?

In the following tutorial, we will explore how to retrieve an Item of our interest, based on several parameters and load it through `xarray`.<br>

This will allow us to seamlessly work with the multi-dimensional array data stored insude `zarr`, opening a new workflow for analysis and visualisation of the EOPF for the Copernicus Sentinel 1, Sentinel 2 and Sentinel 3 missions.